# Poverty Rates for Cities Over 1M
- Joey Ashcroft Code Portion to be merged to master

In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
import gmaps
import json
import os

In [3]:
# Census & gmaps API Keys
from config import (api_key, gkey)

# Configure gmaps
gmaps.configure(api_key=gkey)

In [4]:
#read in data (for master)
county_coordinates = pd.read_csv('county_coordinates.csv')
county_coordinates.head()

,County,State,lat,lng
0,Corozal Municipio,Puerto Rico,18.341062,-66.316836
1,Maunabo Municipio,Puerto Rico,18.007189,-65.899329
2,Peñuelas Municipio,Puerto Rico,18.063358,-66.727390
3,Ponce Municipio,Puerto Rico,18.011077,-66.614062
4,San Sebastián Municipio,Puerto Rico,18.335476,-66.994679


In [5]:
name = "?get=NAME,"
population = "B01003_001E"
poverty_count = "B17001_002E"
county = "for=county:*"

base_url = "https://api.census.gov/data/2017/acs/acs5"
query_url = base_url + name + population + "&" + poverty_count + "&" + county + "&key=" + api_key
query_url

'https://api.census.gov/data/2017/acs/acs5?get=NAME,B01003_001E&B17001_002E&for=county:*&key=e4802ddb92bf7b297f5f10de4176b9d5341c10c8'

In [6]:
census_response = requests.get(query_url)
census_response = census_response.json()

# Convert to DataFrame
poverty_data = pd.DataFrame(census_response)

#replace header with row 1
new_header = poverty_data.iloc[0] #grab the first row for the header
poverty_data = poverty_data[1:] #take the data less the header row
poverty_data.columns = new_header #set the header row as the df header

# Column Reordering
poverty_data = poverty_data.rename(columns={"B01003_001E": "Population",
                                            "B17001_002E": "Poverty Count",
                                            "NAME": "Name", 
                                            "state": "State",
                                            "county":"County"})

# Add in Poverty Rate (Poverty Count / Population)
poverty_data["Poverty Rate"] = 100 * poverty_data["Poverty Count"].astype(int) / poverty_data["Population"].astype(int)

poverty_data.head()

,Name,Population,Poverty Count,State,County,Poverty Rate
1,"Corozal Municipio, Puerto Rico",34933,19146,72,047,54.807775
2,"Maunabo Municipio, Puerto Rico",11297,5945,72,095,52.624591
3,"Peñuelas Municipio, Puerto Rico",21661,12379,72,111,57.148793
4,"Ponce Municipio, Puerto Rico",148863,75187,72,113,50.507514
5,"San Sebastián Municipio, Puerto Rico",38970,21472,72,131,55.098794


In [7]:
poverty_data["County"] = poverty_data["Name"].str.split(pat=",", expand=True)[0]
poverty_data["State"] = poverty_data["Name"].str.split(pat=",", expand=True)[1]
# Parse Final DataFrame
poverty_data = poverty_data[["County","State","Population","Poverty Rate"]]

poverty_data.to_csv("county_poverty_rates.csv", index=False)
poverty_data.head()

,County,State,Population,Poverty Rate
1,Corozal Municipio,Puerto Rico,34933,54.807775
2,Maunabo Municipio,Puerto Rico,11297,52.624591
3,Peñuelas Municipio,Puerto Rico,21661,57.148793
4,Ponce Municipio,Puerto Rico,148863,50.507514
5,San Sebastián Municipio,Puerto Rico,38970,55.098794


### Geocode to get lat/long centroids of counties

In [8]:
#ONLY RUN THIS CODE ONCE AND EXPORT CSV SO DON'T HAVE TO DO AGAIN

# # Target county
# target_county = poverty_data["County"]
# state = poverty_data['State']
# coordinates=[]

# for county, state in zip(target_county, state):
#     # Build the endpoint URL
#     target_url = (f'https://maps.googleapis.com/maps/api/geocode/json?address={county}&key={gkey}')
#     # Run a request to endpoint and convert result to json
#     geo_data = requests.get(target_url).json()

#     try:
#         coordinates.append([county,state,geo_data['results'][0]['geometry']['location']])
#         print(f"{county} County Coordinates Found: {geo_data['results'][0]['geometry']['location']}... Adding to List")
        
#     except IndexError:
#         print(f"{county} County not found... Skipping")

#county_coordinates.to_csv("county_coordinates.csv", index=False)

In [87]:
#DATA FORMATTING: ONLY DONE ONCE BEFORE EXPORTING FILE FOR MASTER

# #converting dictionary inside a list to a data series
# coordinates = pd.DataFrame(coordinates)

# county_coordinates = coordinates[2].apply(pd.Series)
# county_coordinates['County'] = coordinates[0]
# county_coordinates['State'] = coordinates[1]
# county_coordinates = county_coordinates[['County','State','lat','lng']]
# county_coordinates.head()

,County,State,lat,lng
0,Corozal Municipio,Puerto Rico,18.341062,-66.316836
1,Maunabo Municipio,Puerto Rico,18.007189,-65.899329
2,Peñuelas Municipio,Puerto Rico,18.063358,-66.727390
3,Ponce Municipio,Puerto Rico,18.011077,-66.614062
4,San Sebastián Municipio,Puerto Rico,18.335476,-66.994679


## Poverty Heat Map of USA

In [9]:
# Merge two dataframes using an inner join
merge_table = pd.merge(poverty_data, county_coordinates, on=["County","State"])
print(merge_table.shape)
merge_table.head()

(3214, 6)


,County,State,Population,Poverty Rate,lat,lng
0,Corozal Municipio,Puerto Rico,34933,54.807775,18.341062,-66.316836
1,Maunabo Municipio,Puerto Rico,11297,52.624591,18.007189,-65.899329
2,Peñuelas Municipio,Puerto Rico,21661,57.148793,18.063358,-66.727390
3,Ponce Municipio,Puerto Rico,148863,50.507514,18.011077,-66.614062
4,San Sebastián Municipio,Puerto Rico,38970,55.098794,18.335476,-66.994679


In [10]:
# Store 'Lat' and 'Lng' into  locations 
locations = merge_table[["lat", "lng"]].astype(float)

# Convert Poverty Rate to float and store
poverty_rate = merge_table["Poverty Rate"].astype(float)

In [20]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=poverty_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

# Adjust heat_layer setting to help with heatmap dissipating on zoom
heat_layer.dissipating = False
heat_layer.max_intensity = 100
heat_layer.point_radius = 1

fig.add_layer(heat_layer)

plt.savefig("Poverty Heatmap.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

## Homeless Rate

In [12]:
url = "https://api.census.gov/data/2010/dec/sf1?get=PCT020B027,PCT020D027,PCT020E027,PCT021093,PCT021061,PCT021029,PCT021190,PCT021126,PCT021158,PCT020H027,PCT020A027,PCT020I027,PCT020C027,PCT020027,PCT020F027,PCT020G027,NAME&for=county:*&key=bdb18cf93fde53cedda10d62ee24bc6d29e147e5"
response = requests.get(url)
response_json = response.json()

census_pd = pd.DataFrame(response_json)

census_pd.columns = census_pd.iloc[0]
census_pd = census_pd[1:]

census_pd["County"] = census_pd["NAME"].str.split(pat=",", expand=True)[0]
census_pd["State"] = census_pd["NAME"].str.split(pat=",", expand=True)[1]

del census_pd["state"]
del census_pd["county"]
del census_pd["NAME"]

cols = census_pd.columns.drop(["County","State"])

census_pd[cols] = census_pd[cols].apply(pd.to_numeric)

census_pd["Homeless Pop"] = census_pd.sum(axis=1)

final_data = census_pd[["County", "State", "Homeless Pop"]]

# Merge two dataframes using an inner join
merge_table_all = pd.merge(final_data, merge_table, on=["County","State"])
print(merge_table_all.shape)
merge_table_all.head()

(3193, 7)


,County,State,Homeless Pop,Population,Poverty Rate,lat,lng
0,Autauga County,Alabama,0,55036,13.596555,32.579182,-86.499655
1,Bibb County,Alabama,0,22580,13.875111,32.786573,-83.719914
2,Chambers County,Alabama,0,33895,18.551409,29.752496,-94.686378
3,Cherokee County,Alabama,0,25855,15.873139,34.251453,-84.480261
4,Clarke County,Alabama,0,24625,24.804061,33.951935,-83.357567


In [13]:
# Store 'Lat' and 'Lng' into  locations 
locations2 = merge_table_all[["lat", "lng"]].astype(float)

# Convert Poverty Rate to float and store
homeless_count = merge_table_all["Homeless Pop"].tolist()

In [19]:
homeless_layer = gmaps.symbol_layer(
    locations2, fill_color='rgba(0, 0, 0, 0.4)',
    stroke_color='rgba(0, 0, 0, 0.4)', scale=2,
    info_box_content=[f"Homeless Count: {i}" for i in homeless_count])

fig = gmaps.figure()
fig.add_layer(homeless_layer)

plt.savefig("Homeless Heatmap.png")
fig





<Figure size 432x288 with 0 Axes>

In [21]:
# Create a combined map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(homeless_layer)


plt.savefig("Combined Heatmap.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Poverty/Homeless Analysis

In [17]:
print(merge_table_all.shape)
merge_table_all.head()

(3193, 7)


,County,State,Homeless Pop,Population,Poverty Rate,lat,lng
0,Autauga County,Alabama,0,55036,13.596555,32.579182,-86.499655
1,Bibb County,Alabama,0,22580,13.875111,32.786573,-83.719914
2,Chambers County,Alabama,0,33895,18.551409,29.752496,-94.686378
3,Cherokee County,Alabama,0,25855,15.873139,34.251453,-84.480261
4,Clarke County,Alabama,0,24625,24.804061,33.951935,-83.357567


In [18]:
#quick summary of data as a whole

#total number of counties
counties_num=merge_table_all.shape[0]

#average homeless rate
homeless_rate=

#total homeless population in US
homeless_tot=merge_table_all['Homeless Pop'].sum()

#average poverty rate
avg_poverty=round(merge_table_all['Poverty Rate'].mean(),2)

#put data into pandas data frame table
data_summary = pd.DataFrame([{
    'Total Counties': counties_num,'Total States': num_states,'Total Homeless Population': homeless_tot,
    'Average Poverty Rate': avg_poverty
}])

#display
data_summary

SyntaxError: invalid syntax (<ipython-input-18-ca0d6022280f>, line 7)